### INST447 Project Part 2
#### Members: Alexander Chui and Matthew Chan

### What factors affect ratings for an anime?

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing the required libraries
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import numpy as np
from requests import get

In [3]:
#libraries imported for handling rate limits and console formatting
from time import time, sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

In [4]:
import regex as re

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis

import nltk; 

C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
#Connecting to the URL on the web to access its content (Crunchyroll)
# html=urlopen('https://www.crunchyroll.com/fullmetal-alchemist-brotherhood/reviews/helpful/page1')
# html.status

In [7]:
#Connecting to the URL on the web to access its content (MAL)
html=urlopen('https://myanimelist.net/anime/40748/Jujutsu_Kaisen_TV/reviews?p=1')
html.status

200

In [8]:
#Creating an instance of beautifulsoup object to begin web scraping
bs=BeautifulSoup(html.read(),'html.parser')

In [9]:
#Acessing the container objects with the anime info 
anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})
print(len(anime_review_containers))

20


In [10]:
#Creating empty lists to store all the items 
review = []

In [11]:
# Preparing the monitoring of the loop
start_time = time()
request = 0

In [12]:
#Extracting each job name in loop
for page in range(1,14):
    # Make a get request
        #https://myanimelist.net/anime/genre/1/Action?page=1
        #https://myanimelist.net/anime/genre/1/Action?page=2
        response = get('https://myanimelist.net/anime/40748/Jujutsu_Kaisen_TV/reviews?p='+ str(page))
        
#         # Pause the loop
#         sleep(randint(8,15))

#         # Monitor the requests
#         request += 1
#         elapsed_time = time() - start_time
#         print('Request:{}; Frequency: {} requests/s'.format(request, 
#                                                             request/elapsed_time))
#         clear_output(wait = True)

#         # Throw a warning for non-200 status codes
#         if response.status_code != 200:
#             warn('Request: {}; Status code: {}'.format(request, 
#                                                        response.status_code))

#         # Break the loop if the number of requests is greater than expected
#         if request > 100:
#             warn('Number of requests was greater than expected.')
#             break
        
        # Parse the content of the request with BeautifulSoup
        bs = BeautifulSoup(response.text, 'html.parser')
        
        #Acessing the container objects with the anime info 
        anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})

        #Extracting all the elements 
        for container in anime_review_containers:
        # If the movie has Metascore, then extract:
        #  if container.find('div', {'class':{'scoremem'}})  is not None:
        
        # The name of anime
            try: 
                reviews = container.find('div', {'class':'spaceit textReadability word-break pt8 mt8'})
                review.append(reviews.text)
            except:
                review.append('Missing')

In [13]:
#Creating a pandas dataframe to store the objects extracted
anime_review = pd.DataFrame({'Review': review
})
print(anime_review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  260 non-null    object
dtypes: object(1)
memory usage: 2.2+ KB
None


In [14]:
# Display the top 5 rows of dataset
anime_review.head()

,Review
0,\n\n\n\nOverall\n6\n\n\nStory\n4\n\n\nAnimatio...
1,\n\n\n\nOverall\n9\n\n\nStory\n0\n\n\nAnimatio...
2,\n\n\n\nOverall\n9\n\n\nStory\n9\n\n\nAnimatio...
3,\n\n\n\nOverall\n5\n\n\nStory\n1\n\n\nAnimatio...
4,\n\n\n\nOverall\n8\n\n\nStory\n7\n\n\nAnimatio...


In [15]:
# Convert to list
data = anime_review['Review'].tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

print(data[0])

<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-15-b412c0db5fd7>:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


 Overall 6 Story 4 Animation 9 Sound 7 Character 5 Enjoyment 7 “It’s not about whether I can, I have to do it!” – Megumi Fushiguro...and also me steeling myself to post a critical review on this beloved anime. Jujutsu Kaisen is the latest shounen to gain a surge of popularity and get invited to join the mainstream table of the true anime titans, probably taking a seat right next to Demon Slayer. And I can say with certainty that this is primarily because of Studio Mappa's admirable dedication to making this show, particularly the fight scenes, look breathtakingly good. But here's the age old question: Does fantastic animation trump average writing? Yeah, no. Heh, sorry to be blunt. I guess I'm taking a lesson out of protagonist Itadori's book. But who's to say that Jujutsu doesn't have a bit of both? While I find there to be several glaring issues with the anime, it also has many great aspects. So therefore, I'm gonna do things a little differently. This review will go over 5 areas tha

In [16]:
# Clean and tokenize the reviews

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'it', 'not', 'about', 'whether', 'can', 'have', 'to', 'do', 'it', 'megumi', 'fushiguro', 'and', 'also', 'me', 'steeling', 'myself', 'to', 'post', 'critical', 'review', 'on', 'this', 'beloved', 'anime', 'jujutsu', 'kaisen', 'is', 'the', 'latest', 'shounen', 'to', 'gain', 'surge', 'of', 'popularity', 'and', 'get', 'invited', 'to', 'join', 'the', 'mainstream', 'table', 'of', 'the', 'true', 'anime', 'titans', 'probably', 'taking', 'seat', 'right', 'next', 'to', 'demon', 'slayer', 'and', 'can', 'say', 'with', 'certainty', 'that', 'this', 'is', 'primarily', 'because', 'of', 'studio', 'mappa', 'admirable', 'dedication', 'to', 'making', 'this', 'show', 'particularly', 'the', 'fight', 'scenes', 'look', 'breathtakingly', 'good', 'but', 'here', 'the', 'age', 'old', 'question', 'does', 'fantastic', 'animation', 'trump', 'average', 'writing', 'yeah', 'no', 'heh', 'sorry', 'to', 'be', 'blunt', 'guess', 'taking', 'lesson', 'out', 'o

In [17]:
# Create bigrams, remove stop words and lemmatize the reviews

# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'the', 'thing', 'about', 'trends', 'that', 'lot', 'of', 'people', 'take', 'for', 'granted', 'is', 'that', 'the', 'more', 'overstuffed', 'particular', 'set', 'of', 'ideas', 'are', 'the', 'more', 'likely', 'it', 'is', 'to', 'see', 'either', 'flipped', 'on', 'its', 'head', 'see', 'shrek', 'for', 'fairy', 'tale', 'musicals', 'or', 'given', 'more', 'offbeat', 'renditions', 'see', 'joker', 'or', 'into', 'the', 'spider', 'verse', 'for', 'superhero', 'blockbusters', 'genres', 'being', 'overstuffed', 'if', 'anything', 'should', 'encourage', 'more', 'experimentation', 'and', 'refinement', 'over', 'the', 'years', 'the', 'shonen', 'genre', 'has', 'gone', 'through', 'number', 'of', 'these', 'phases', 'from', 'past', 'its', 'formative', 'years', 'with', 'dragon', 'ball', 'and', 'dragon', 'ball', 'you', 've', 'gotten', 'shows', 'using', 'the', 'mold', 'for', 'massive', 'tapestries', 'one', 'piece', 'those', 'attempting', 'to', 'rewr

In [18]:
# Defining removing stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Defining making bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Defining lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'megumi', 'fushiguro', 'also', 'steel', 'post', 'critical', 'review', 'beloved', 'anime', 'late', 'shounen', 'gain', 'surge', 'popularity', 'get', 'invite', 'join', 'mainstream', 'table', 'true', 'anime', 'titan', 'probably', 'take', 'seat', 'right', 'next', 'say', 'certainty', 'primarily', 'studio', 'mappa', 'admirable', 'dedication', 'making', 'show', 'particularly', 'fight_scene', 'look', 'breathtakingly', 'good', 'age', 'old', 'question', 'fantastic', 'animation', 'trump', 'average', 'writing', 'blunt', 'guess', 'take', 'lesson', 'protagonist', 'itadori', 'book', 'say', 'jujutsu', 'bit', 'find', 'several', 'glaring', 'issue', 'anime', 'also', 'many', 'great', 'aspect', 'therefore', 'gon', 'thing', 'little', 'differently', 'review', 'go', 'area', 'believe', 'anime', 'handle', 'poorly', 'represent', 'sukuna', 'king', 'curse', 'curse', 'finger', 'department', 'triumphant', 'success', 'eye', 'represent', 'sukuna', 'fi

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 4), (2, 3), (3, 3), (4, 3), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 5), (13, 1), (14, 1), (15, 3), (16, 10), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 3), (34, 3), (35, 4), (36, 1), (37, 3), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 20), (71, 1), (72, 1), (73, 1), (74, 1), (75, 5), (76, 2), (77, 5), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 5), (84, 3), (85, 1), (86, 1), (87, 3), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 14), (95, 1), (96, 2), (97, 1), (98, 1), (99, 3), (100, 1), (101, 3), (102, 1), (103, 1), (104, 3), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 

In [21]:
# Using LDA to develop a topic model with 3 models

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 3 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.031*"character" + 0.017*"anime" + 0.017*"story" + 0.015*"good" + 0.013*"show" + 0.013*"animation" + 0.010*"well" + 0.010*"sound" + 0.009*"overall" + 0.009*"really"'), (1, '0.014*"naruto" + 0.011*"fight" + 0.005*"anime" + 0.005*"kill" + 0.004*"shounen" + 0.004*"haired" + 0.004*"yoshino" + 0.004*"hero_academia" + 0.004*"girl" + 0.003*"pretend"'), (2, '0.011*"make" + 0.010*"character" + 0.008*"anime" + 0.008*"strong" + 0.007*"show" + 0.007*"curse" + 0.007*"fight" + 0.006*"say" + 0.006*"bad" + 0.005*"get"'), (3, '0.161*"epic_epic" + 0.002*"gaming" + 0.002*"space" + 0.002*"crisp" + 0.002*"consistently" + 0.002*"previous" + 0.002*"epic" + 0.002*"every_single" + 0.002*"fill" + 0.002*"masterpiece"'), (4, '0.000*"epic_epic" + 0.000*"character" + 0.000*"anime" + 0.000*"get" + 0.000*"show" + 0.000*"see" + 0.000*"well" + 0.000*"animation" + 0.000*"make" + 0.000*"episode"'), (5, '0.002*"mega" + 0.001*"naru" + 0.001*"fathom" + 0.001*"gentle" + 0.001*"hoping" + 0.001*"assign" + 0.001*"curve" 

In [22]:
# Generate a visualization of the topic models

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.281602  0.076442       1        1  83.804101
2      0.153980 -0.094186       2        1  12.149925
7     -0.111537  0.139388       3        1   2.231321
1     -0.072226 -0.076096       4        1   1.582093
3     -0.078232 -0.019098       5        1   0.154208
5     -0.060092 -0.009660       6        1   0.064119
6     -0.056748 -0.008395       7        1   0.007119
4     -0.056748 -0.008396       8        1   0.007115, topic_info=        Term        Freq       Total Category  logprob  loglift
157    fight  294.000000  294.000000  Default   30.000  30.0000
16     anime  891.000000  891.000000  Default   29.000  29.0000
396  shounen  327.000000  327.000000  Default   28.000  28.0000
275     make  430.000000  430.000000  Default   27.000  27.0000
606   naruto   91.000000   91.000000  Default   26.000  26.0000
..       ...         ...         ...      ...      ...      ...
234  itadori    0.000669  175.803703   Topic8   -8.697  -2.9289
483     want    0.000669  174.850276   Topic8   -8.697  -2.9235
182     give    0.000669  252.944376   Topic8   -8.697  -3.2927
184       go    0.000669  306.296475   Topic8   -8.697  -3.4841
155     feel    0.000669  268.794564   Topic8   -8.697  -3.3535

[463 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
5762      4  0.606495  abomination
3         1  0.820620       action
3         2  0.175278       action
502       4  0.978854          ali
12        1  0.957121         also
...     ...       ...          ...
3440      4  0.994389         wind
491       1  0.427729         word
491       2  0.549937         word
2255      2  0.883523       writer
2060      4  0.801112      yoshino

[367 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 8, 2, 4, 6, 7, 5])

In [23]:
# Save the data frame as a CSV file
# job_datas.to_csv(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Programming Assignment 2\Chan_Matthew_INST447_PA2.csv')

In [24]:
# Week 9 Topic Modelling
# ner_corpus = ' '.join(ner_df['Name'])
# corp = open(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Week 9\corpus.txt',"w")
# corp.writelines(ner_corpus)
# corp.close()